<a href="https://colab.research.google.com/github/ElanElanElanElan/DiffusingColab/blob/main/DiffusingGradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Installing** and **importing** all neccesary and semi neccesary stuff
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import shutil
import time
from PIL import Image
!pip install --upgrade gradio diffusers accelerate transformers xformers safetensors
import gradio as gr
import torch
from diffusers import DiffusionPipeline,StableDiffusionPipeline,StableDiffusionXLPipeline,DPMSolverMultistepScheduler,AutoencoderTiny

In [ ]:
#@title use drive and create the setup folders
from google.colab import drive
drive.mount('/content/drive')
!mkdir Loras
!mkdir Outputs
!mkdir Plots
lora_folder = '/content/Loras'
output_folder = '/content/Outputs'
plot_folder = "/content/Plots"

In [ ]:
#@title select which useful base model and optimization to use
Enter_model = "Sdxl" # @param ["Sdxl", "1.5"]
if Enter_model == 'Sdxl':
    model_id = "stabilityai/stable-diffusion-xl-base-1.0"
    Pipeline =  StableDiffusionXLPipeline.from_pretrained(model_id, torch_dtype=torch.float16,variant="fp16", safety_checker=None, use_safetensors=True)
    Pipeline.scheduler = DPMSolverMultistepScheduler.from_config(Pipeline.scheduler.config)
else:
    model_id = "runwayml/stable-diffusion-v1-5"
    Pipeline =  StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16,variant="fp16", safety_checker=None, use_safetensors=True)
    Pipeline.scheduler = DPMSolverMultistepScheduler.from_config(Pipeline.scheduler.config)

enter_optimisation = "optimised" #@param ["none",'optimised']
if enter_optimisation == "optimised":
    Pipeline.vae = AutoencoderTiny.from_pretrained("madebyollin/taesdxl", torch_dtype=torch.float16)
    Pipeline.enable_attention_slicing()
    Pipeline.enable_vae_slicing()
    Pipeline.enable_vae_tiling()
    Pipeline.to('cuda')


run all the def function cells below

In [ ]:
def basic_generator(input_prompt: str, number_of_pics: int):
    number_of_pics = int(number_of_pics)
    #batch_size= int(batch_size)

    output = [Pipeline(prompt = input_prompt, num_inference_steps= 20).images for i in range(number_of_pics)]
    for index, image in enumerate(output):
        file_name = f"image{index}.jpg"
        image[0].save(os.path.join(output_folder,file_name))
    return output

In [ ]:
def Loading_Lora(lora_name_string):
    os.listdir(lora_folder)
    if lora_name_string in lora_folder:
        Pipeline.load_lora_weights(os.path.join(lora_folder, lora_name_string))
        return Pipeline

In [ ]:
def plot_images(images:list):
    image_paths = [os.path.join(output_folder, f"image{i}.jpg" ) for i in range(len(images))]
    num_images = len(images)
    num_rows = int(num_images ** 0.5)
    num_cols = (num_images + num_rows - 1) // num_rows
    fig, axes = plt.subplots(num_rows, num_cols)
    for i, ax in enumerate(axes.ravel()):
        if i < len(image_paths):
            img = Image.open(image_paths[i])
            ax.imshow(img)
            ax.axis('off')

    i = len(os.listdir(plot_folder))
    while True:
        plt_name = os.path.join(plot_folder, f"plot{i}.jpg")
        if plt_name in os.listdir(plot_folder):
            plt_name = os.path.join(plot_folder, f"plot{i+1}.jpg")
            i += 1
        else:
            break

    plt.savefig(plt_name, bbox_inches="tight")
    plt.close()
    for image_path in image_paths:
       os.remove(image_path)


In [ ]:
def Lora_Comparision()

Use the *Code Cell below* to run the pipeline using the **functions** defined in the *above code cell*
For example-

1.  **`basic_generator("anime waifu", 2)`** will get you 2 individual anime waifus pics in the output folder
2.  **`plot_images(basic_generator("anime waifus",12))`** will you get you your 12 anime waifus on a single image in the plot directory



In [ ]:
plot_images(basic_generator("batman by the beach",2))

run the **`Gradio`** interface (not completed)

In [ ]:
#Iface = gr.Interface(fn=user_inputs, inputs= ["text","number"], outputs= "image")
#Iface.launch(debug=True)